# **Desenvolvimento de Modelo com Tidyverse**

Na análise de dados, muitas vezes usamos o desenvolvimento de modelos para nos ajudar a prever observações futuras a partir dos dados que temos. Um Modelo nos ajudará a entender a relação exata entre diferentes variáveis e como essas variáveis são usadas para prever o resultado.

## **Carregar bibliotecas e dados**

A biblioteca Tidyverse é uma ferramenta útil que fornece várias ferramentas para visualização de dados, manipulação de dados e leitura de vários conjuntos de dados em um quadro de dados.

In [1]:
# Carregar o Tidyverse
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::%||%()   masks base::%||%()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# url onde os dados estão localizados
url <- "https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/lax_to_jfk.tar.gz"

# baixe o arquivo
download.file(url, destfile = "lax_to_jfk.tar.gz")

# se você executar isso em sua máquina local, poderá remover tar = "internal"
untar("lax_to_jfk.tar.gz", tar = "internal")

# somente read_csv
sub_airline <- read_csv("lax_to_jfk/lax_to_jfk.csv",
                     col_types = cols('DivDistance' = col_number(),
                                      'DivArrDelay' = col_number()))

Warning message in untar2(tarfile, files, list, exdir, restore_times):
“using pax extended headers”


**Observação:** O projeto GitHub tem uma pasta chamada Files, nela contem esses dados já baixado

## **Regressão Linear Simples**

A Regressão Linear Simples é um método para nos ajudar a entender a relação entre duas variáveis:

*   $X$: A variável preditora/independente
*   $Y$: A variável resposta/dependente (que queremos prever)

O resultado da regressão linear é uma função linear que prediz a variável resposta (dependente) em função da variável preditora (independente).

**Função do modelo linear:**
$$
\hat{Y} = b_0 + b_1  X
$$



*   $b_0$: refere-se ao intercepto da reta de regressão, em outras palavras: o valor de Y quando X é 0
*   $b_1$: refere-se à inclinação da reta de regressão, em outras palavras: o valor com o qual Y muda quando X aumenta em 1 unidade
*   $\hat{Y}$: (y-hat) é o valor previsto a partir do modelo linear

### **Ajustar os dados em um modelo de regressão linear**

Primeiro, vamos apenas olhar para os dados da Alaska Airline (AA) e CarrierDelay.

In [3]:
# Defina o conjunto de dados apenas com AA como Reporting_Airline
aa_delays <- sub_airline %>%
  filter(CarrierDelay != "NA", Reporting_Airline == "AA")

head(aa_delays)

Month,DayOfWeek,FlightDate,Reporting_Airline,Origin,Dest,CRSDepTime,CRSArrTime,DepTime,ArrTime,⋯,ArrDelayMinutes,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DepDelay,DepDelayMinutes,DivDistance,DivArrDelay
<dbl>,<dbl>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6,4,2006-06-01,AA,LAX,JFK,1515,2332,1507,2353,⋯,21,0,0,21,0,0,-8,0,NA,NA
6,5,2013-06-28,AA,LAX,JFK,1200,2045,1328,2220,⋯,95,5,0,7,0,83,88,88,NA,NA
10,3,2005-10-12,AA,LAX,JFK,0930,1755,0958,1823,⋯,28,0,0,28,0,0,28,28,NA,NA
4,6,2011-04-23,AA,LAX,JFK,0925,1800,1040,1850,⋯,50,0,0,50,0,0,75,75,NA,NA
8,7,2011-08-14,AA,LAX,JFK,1100,1945,1200,2042,⋯,57,0,0,57,0,0,60,60,NA,NA
2,5,2004-02-06,AA,LAX,JFK,1500,2306,1607,2348,⋯,42,0,0,0,0,42,67,67,NA,NA


Para este exemplo, queremos ver como o atraso de partida (DepDelayMinutes) pode nos ajudar a prever o atraso de chegada (ArrDelayMinutes). Usando regressão linear simples, criaremos uma função linear com "DepDelayMinutes" como variável preditora e "ArrDelayMinutes" como variável resposta.

In [4]:
linear_model <- lm(ArrDelayMinutes ~ DepDelayMinutes, data = aa_delays)

In [5]:
# Resuma o modelo de regressão usando
summary(linear_model)


Call:
lm(formula = ArrDelayMinutes ~ DepDelayMinutes, data = aa_delays)

Residuals:
    Min      1Q  Median      3Q     Max 
-37.234 -12.716  -1.354   7.747  93.646 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)      17.3544     2.5084   6.919  2.9e-10 ***
DepDelayMinutes   0.7523     0.0399  18.855  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 20.03 on 113 degrees of freedom
Multiple R-squared:  0.7588,	Adjusted R-squared:  0.7567 
F-statistic: 355.5 on 1 and 113 DF,  p-value: < 2.2e-16


Podemos produzir uma previsão de três novos pontos de dados.

In [6]:
# Dados de entrada que usamos para prever
new_depdelay <- data.frame(
  DepDelayMinutes = c(12, 19, 24))

# Preveja os pontos de dados
pred <- predict(linear_model, newdata = new_depdelay, interval = "confidence")
pred

,fit,lwr,upr
1,26.38175,21.98838,30.77512
2,31.64769,27.52630,35.76908
3,35.40907,31.44593,39.37222


Quando imprimimos o objeto pred, podemos ver que existem 3 colunas: fit, lwr e upr. A coluna "ajuste" é a previsão dos resultados das entradas. E "lwr" e "upr" são o limite inferior e o limite superior dos intervalos de confiança de 95% dos resultados da predição. O intervalo de confiança reflete a incerteza em torno das previsões médias.

### **Qual o valor da interceptação e a inclinação?**

Usando o modelo ajustado, linear_model, você pode pegar os coeficientes de atributo usando $.

In [7]:
linear_model$coefficients

(Intercept) DepDelayMinutes 
     17.3544286       0.7522769

### **Qual é o modelo linear final estimado que temos?**

Criando uma função linear com "CarrierDelay" como a variável preditora e "ArrDelayMinutes" como a variável de resposta. Em seguida, encontrando os coeficientes (intercepto e inclinação) do modelo.

In [8]:
linear_model2 <- lm(ArrDelayMinutes ~ CarrierDelay, data = aa_delays)

linear_model2$coefficients

(Intercept) CarrierDelay 
  35.1176108    0.7031761

**A equação da reta prevista. Usando x e yhat ou 'CarrierDelay' ou 'ArrDelayMinutes':**

Yhat = 35.11 + 0.7032 * X

ArrDelayMinutes = 35.11 + 0.7032 * CarrierDelay  

## **Regressão Linear Múltipla**

E se quisermos prever minutos de atraso de chegada usando mais de uma variável?

Se quisermos usar mais variáveis em nosso modelo para prever minutos de atraso de chegada, podemos usar a Regressão Linear Múltipla. A Regressão Linear Múltipla é muito semelhante à Regressão Linear Simples, mas esse método é usado para explicar a relação entre uma variável de resposta contínua (dependente) e duas ou mais variáveis preditoras (independentes).



*   $Y$: Variável de resposta
*   $X_1$: Variável preditora 1
*   $X_2$: Variável preditora 2

$$
\hat{Y} = b_0 + b_1 X_1 + b_2 X_2
$$

onde:

*  $b_0$: interceptar
*  $b_1$: coeficiente da variável 1
*  $b_2$: coeficiente da variável 2

Pelo projeto anterior ([link](https://github.com/Code-Lipe/exploratory_data_analysis_tidyverse)), sabemos que outros bons preditores de ArrDelayMinutes podem ser:

*   DepDelayMinutes
*   AtrasoAeronaveAtraso

Vamos desenvolver um modelo usando essas variáveis como variáveis preditoras, ajustando os dados.

In [9]:
mlr <- lm(ArrDelayMinutes ~ DepDelayMinutes + LateAircraftDelay, data = aa_delays)

summary(mlr)


Call:
lm(formula = ArrDelayMinutes ~ DepDelayMinutes + LateAircraftDelay, 
    data = aa_delays)

Residuals:
    Min      1Q  Median      3Q     Max 
-37.188 -12.545  -1.317   7.791  93.683 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)       17.31707    2.53786   6.823 4.78e-10 ***
DepDelayMinutes    0.75556    0.04822  15.668  < 2e-16 ***
LateAircraftDelay -0.01028    0.08407  -0.122    0.903    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 20.11 on 112 degrees of freedom
Multiple R-squared:  0.7588,	Adjusted R-squared:  0.7545 
F-statistic: 176.2 on 2 and 112 DF,  p-value: < 2.2e-16


Qual o valor da interceptação ($b_0$) e os coeficientes ($b_1$, $b_2$)?

In [10]:
mlr$coefficients

(Intercept)   DepDelayMinutes LateAircraftDelay 
      17.31706644        0.75555543       -0.01027513